In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import numpy as np
import tensorflow as tf
import pickle

In [6]:
# Carrega os dados
df = pd.read_csv(r'..\..\data\Tweets_Mg.csv', encoding='utf-8')

In [8]:
# Pré-processamento dos dados
df = df[["Text", "Classificacao"]]  # Mantém apenas as colunas necessárias
df["Classificacao"] = df["Classificacao"].replace({'Negativo': 0, 'Positivo': 1, 'Neutro' : 2})
df.head()


C:\Users\rafael.nsouza\AppData\Local\Temp\ipykernel_16800\3037184820.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Classificacao"] = df["Classificacao"].replace({'Negativo': 0, 'Positivo': 1, 'Neutro' : 2})


,Text,Classificacao
0,���⛪ @ Catedral de Santo Antônio - Governador ...,2
1,"� @ Governador Valadares, Minas Gerais https:/...",2
2,"�� @ Governador Valadares, Minas Gerais https:...",2
3,��� https://t.co/BnDsO34qK0,2
4,��� PSOL vai questionar aumento de vereadores ...,0


In [9]:
# Tokenização e Vetorização
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(df["Text"])
sequences = tokenizer.texts_to_sequences(df["Text"])
padded_sequences = pad_sequences(sequences, maxlen=100)


In [10]:
# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df["Classificacao"], test_size=0.2, random_state=42)


In [11]:
# Construção do modelo RNN
model = Sequential([
    Embedding(input_dim=10000, output_dim=16),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [12]:
# Compilação do modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
# Treinamento do modelo
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))


Epoch 1/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 7s 100ms/step - accuracy: 0.4077 - loss: 0.3396 - val_accuracy: 0.4030 - val_loss: -0.3763
Epoch 2/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 95ms/step - accuracy: 0.4929 - loss: -2.1015 - val_accuracy: 0.6476 - val_loss: -18.2630
Epoch 3/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 82ms/step - accuracy: 0.6404 - loss: -23.8340 - val_accuracy: 0.6500 - val_loss: -58.8776
Epoch 4/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 82ms/step - accuracy: 0.6456 - loss: -66.5308 - val_accuracy: 0.6683 - val_loss: -105.8900
Epoch 5/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 82ms/step - accuracy: 0.6644 - loss: -112.6818 - val_accuracy: 0.6683 - val_loss: -176.9559
Epoch 6/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.6683 - loss: -190.6640 - val_accuracy: 0.6720 - val_loss: -271.2330
Epoch 7/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 82ms/step - accuracy: 0.6780 - loss: -285.7560 - val_accuracy: 0.6683 - val_loss: -373.8411
Epoch 8/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - accuracy: 0.6874 - loss: -

In [14]:
# Avaliação do modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Acurácia do modelo nos dados de teste:", test_acc)

52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6935 - loss: -672.8414
Acurácia do modelo nos dados de teste: 0.6762195229530334


In [ ]:
# Salvar o modelo e o tokenizer
model.save(r"..\modelos\modelo_rnn.keras")
with open(r"..\modelos\tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Modelo treinado e tokenizer salvos com sucesso.")

In [15]:
test_sentences = [
    "O dia está lindo e ensolarado, é um ótimo momento para um passeio no parque.",
    "Estou muito feliz com o resultado do meu trabalho hoje, me sinto realizado.",
    "Infelizmente, meu time perdeu o jogo ontem à noite, estou muito desapontado.",
    "Acabei de assistir a um filme incrível, recomendo a todos!",
    "O trânsito estava terrível esta manhã, cheguei atrasado no trabalho."
]

In [16]:
# Tokenização das frases
test_sequences = tokenizer.texts_to_sequences(test_sentences)
padded_test_sequences = pad_sequences(test_sequences, maxlen=100)

# Fazendo previsões
predictions = model.predict(padded_test_sequences)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step


In [17]:
# Arredondando as previsões para obter classes binárias (0 ou 1)
binary_predictions = np.round(predictions).flatten()

In [18]:
# Mapeando as classes de volta para as strings originais
class_mapping = {0: 'Negativo', 1: 'Positivo', 2: 'Neutro'}
predicted_classes = [class_mapping[int(pred)] for pred in binary_predictions]

In [19]:
# Exibindo as previsões
for sentence, prediction in zip(test_sentences, predicted_classes):
    print(f"Frase: {sentence}")
    print(f"Previsão: {prediction}\n")

Frase: O dia está lindo e ensolarado, é um ótimo momento para um passeio no parque.
Previsão: Positivo

Frase: Estou muito feliz com o resultado do meu trabalho hoje, me sinto realizado.
Previsão: Positivo

Frase: Infelizmente, meu time perdeu o jogo ontem à noite, estou muito desapontado.
Previsão: Positivo

Frase: Acabei de assistir a um filme incrível, recomendo a todos!
Previsão: Positivo

Frase: O trânsito estava terrível esta manhã, cheguei atrasado no trabalho.
Previsão: Positivo

